과제W06_20150962_채현종

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.iolib.summary2 import summary_col

import tensorflow as tf
tf.random.set_seed(777)  # for reproducibility

np.set_printoptions(precision=5)
np.set_printoptions(suppress=True)

1.2019년 가계복지데이터를 불러옵니다. 10점

In [3]:
df = pd.read_csv('C:/Users/yssg9187/Desktop/20-2/3. 응용자료분석/D_data/b_koweps01.csv')

2.성별더미, SMOKING DUMMIES, JOB DUMMIES, RELIGION 더미를 만듭니다. 10점

In [4]:
df = df.drop(['Unnamed: 0'], axis=1)

In [5]:
df.head()

,sex,religion,income,education_year,age,marriage_type,job_code,smoking
0,1,1,108.9,16,78,2,7,2
1,2,1,20.0,6,80,3,9,2
2,1,2,322.0,16,42,1,3,1
3,2,2,120.0,16,45,1,2,2
4,1,1,300.0,16,45,3,2,2


In [6]:
sex_dummies = pd.get_dummies(df['sex']).rename(columns=lambda x: 's'+str(x))
smoking_dummies = pd.get_dummies(df['smoking']).rename(columns=lambda x: 'k'+str(x))
job_dummies = pd.get_dummies(df['job_code']).rename(columns=lambda x: 'd'+str(x))
religion_dummies = pd.get_dummies(df['religion']).rename(columns=lambda x: 'r'+str(x))

In [7]:
df.head()

,sex,religion,income,education_year,age,marriage_type,job_code,smoking
0,1,1,108.9,16,78,2,7,2
1,2,1,20.0,6,80,3,9,2
2,1,2,322.0,16,42,1,3,1
3,2,2,120.0,16,45,1,2,2
4,1,1,300.0,16,45,3,2,2


3.가계복지데이터와 더미변수를 합합니다. 10점

In [8]:
df1 = pd.concat([df,sex_dummies, smoking_dummies, job_dummies, religion_dummies], axis=1)

In [9]:
df1.head()

,sex,religion,income,education_year,age,marriage_type,job_code,smoking,s1,s2,...,d3,d4,d5,d6,d7,d8,d9,d10,r1,r2
0,1,1,108.9,16,78,2,7,2,1,0,...,0,0,0,0,1,0,0,0,1,0
1,2,1,20.0,6,80,3,9,2,0,1,...,0,0,0,0,0,0,1,0,1,0
2,1,2,322.0,16,42,1,3,1,1,0,...,1,0,0,0,0,0,0,0,0,1
3,2,2,120.0,16,45,1,2,2,0,1,...,0,0,0,0,0,0,0,0,0,1
4,1,1,300.0,16,45,3,2,2,1,0,...,0,0,0,0,0,0,0,0,1,0


4. 소득, 나이, 교육연수, 성별더미, 흡연여부더미, 종교여부 더미를 X 변수로, 혼인상태(기혼, 별거이혼, 미혼)를 Y 변수로 둡니다 10점

In [10]:
x_data =df1[['income', 'age', 'education_year', 's2', 'k2', 'r2']].to_numpy()
y_data =df1[['marriage_type']].to_numpy()

In [11]:
x_data = x_data.astype(np.float32)

5. 결혼여부를 one-hot encoding 방식으로 바꿉니다. 10점

In [12]:
X = np.asarray(x_data, dtype=np.float32)
y = np.asarray(y_data, dtype=np.float32)

In [13]:
print(X.shape)
print(y.shape)

(4507, 6)
(4507, 1)


In [14]:
k = x_data.shape[1]

In [15]:
k

6

In [16]:
nb_classes = 3

In [17]:
Y_one_hot = tf.one_hot(y.astype(np.int32), nb_classes)
Y_one_hot = tf.reshape(Y_one_hot,[4507, -1] )
Y_one_hot

<tf.Tensor: shape=(4507, 3), dtype=float32, numpy=
array([[0., 0., 1.],
       [0., 0., 0.],
       [0., 1., 0.],
       ...,
       [0., 0., 0.],
       [0., 1., 0.],
       [0., 0., 0.]], dtype=float32)>

6. tensorflow API를 이용하여 dataset을 구축합니다. 10점

In [18]:
dataset = tf.data.Dataset.from_tensor_slices((x_data, Y_one_hot)).batch(len(x_data))
dataset

<BatchDataset shapes: ((None, 6), (None, 3)), types: (tf.float32, tf.float32)>

In [19]:
W = tf.Variable(tf.random.normal((k, nb_classes)), name='weight')
b = tf.Variable(tf.random.normal((nb_classes,)), name='bias')

7. softmax 함수, loss funciton에 대한 함수, 미분과정에 대한 함수를 만듭니다. 10점

In [20]:
learning_rate = 0.001

In [21]:
def softmax_regression(features):
    hypothesis  = tf.nn.softmax(tf.matmul(features, W) + b)
    return hypothesis

def loss_fn(hypothesis, labels):
    logits = tf.matmul(features, W) + b
    cost_i = tf.keras.losses.categorical_crossentropy(y_true=Y_one_hot, y_pred=logits, from_logits=True) 
    cost = tf.reduce_mean(cost_i) 
    return cost

optimizer = tf.optimizers.SGD(learning_rate=0.001)

def grad(features, labels):
    with tf.GradientTape() as tape:
        loss_value = loss_fn(softmax_regression(features),labels)
    return tape.gradient(loss_value, [W,b])


8. 함수를 이용하여 machine learning 방식으로 accuracy를 구합니다. 10점

In [22]:
EPOCHS = 50001

for step in range(EPOCHS):
    for features, labels  in iter(dataset):
        
        hypothesis = softmax_regression(features)
        grads = grad(features, labels)
        optimizer.apply_gradients(grads_and_vars=zip(grads,[W,b]))
        cost = loss_fn(hypothesis,labels)
        
        predicted = tf.math.argmax(hypothesis, 1)
        correct_prediction = tf.equal(predicted, tf.math.argmax(Y_one_hot,1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, dtype=tf.float32))
        
       
        if step % 5000 == 0:
            print("Iter: {}, Loss: {:.4f},  Accuracy: {:.4f}".format(step, cost, accuracy))

Iter: 0, Loss: 25.3525,  Accuracy: 0.2487
Iter: 5000, Loss: 4.0238,  Accuracy: 0.6386
Iter: 10000, Loss: 1.4236,  Accuracy: 0.6443
Iter: 15000, Loss: 0.7906,  Accuracy: 0.6406
Iter: 20000, Loss: 0.7486,  Accuracy: 0.6335
Iter: 25000, Loss: 1.7512,  Accuracy: 0.6264
Iter: 30000, Loss: 2.7604,  Accuracy: 0.3446
Iter: 35000, Loss: 5.3452,  Accuracy: 0.0708
Iter: 40000, Loss: 3.2241,  Accuracy: 0.6441
Iter: 45000, Loss: 1.8197,  Accuracy: 0.3282
Iter: 50000, Loss: 1.1195,  Accuracy: 0.4049


9. 정확도의 추이을 살펴보고 이 모델을 평가합니다. 10점

loss 와 accuracy가 들쭉날쭉하다. 
정상적인 모델이라면 loss가 줄어들며 accuracy가 높아질텐데, 
반복횟수에 상관없이 값이 오르락 내리락한다.
좋은 모델이라고 할 수 없다.